In [1]:
import pylab
import matplotlib
import numpy as np
import pandas as pd
from skimage.util import img_as_float
from skimage.io import imread

Загрузите картинку parrots.jpg. Преобразуйте изображение, приведя все значения в интервал от 0 до 1. Для этого можно воспользоваться функцией img_as_float из модуля skimage. Обратите внимание на этот шаг, так как при работе с исходным изображением вы получите некорректный результат.

In [2]:
image = img_as_float(imread('parrots.jpg'))
w, h, d = image.shape

In [3]:
for mass in image:
    for arr in mass:
        for num in arr:
            if num > 1:
                print 'IT\'S a TRAP!'

Создайте матрицу объекты-признаки: характеризуйте каждый пиксель тремя координатами - значениями интенсивности в пространстве RGB.

In [4]:
pixels = pd.DataFrame(np.reshape(image, (w*h, d)), columns=['R', 'G', 'B'])
pixels.head()

,R,G,B
0,0.015686,0.494118,0.019608
1,0.007843,0.494118,0.007843
2,0.007843,0.494118,0.007843
3,0.007843,0.494118,0.007843
4,0.007843,0.501961,0.011765


Запустите алгоритм K-Means с параметрами init='k-means++' и random_state=241. После выделения кластеров все пиксели, отнесенные в один кластер, попробуйте заполнить двумя способами: медианным и средним цветом по кластеру.

In [5]:
from sklearn.cluster import KMeans
%matplotlib inline

In [6]:
def search(n):
    kmean = KMeans(n_clusters=n,init='k-means++', random_state=241).fit(pixels)
    pixels['Label']= kmean.labels_
    
    medians, means = pixels.groupby('Label').median().values, pixels.groupby('Label').mean().values
    
    medians_pixels, means_pixels = [medians[c] for c in pixels['Label'].values], [means[c] for c in pixels['Label'].values]
    
    medians_img, means_img = np.reshape(medians_pixels, (w,h,d)), np.reshape(means_pixels, (w,h,d))
      
    return means_img, medians_img

In [7]:
from math import log10
def psnr(img1,img2):
    mse = np.mean((img1-img2)**2)
    return 10 * log10(1/mse)

Найдите минимальное количество кластеров, при котором значение PSNR выше 20 (можно рассмотреть не более 20 кластеров, но не забудьте рассмотреть оба способа заполнения пикселей одного кластера). Это число и будет ответом в данной задаче.


In [12]:
from skimage.measure import compare_psnr
for i in range(1, 21):
    tmp = search(i)
    print i, psnr(image,tmp[0]), psnr(image,tmp[1])
    #print i, compare_psnr(image, tmp[0]), compare_psnr(image, tmp[1])
    if psnr(image,tmp[0])>20 or psnr(image,tmp[1])>20:
        break

1 9.81883554477 9.42784046051
2 12.0801770849 11.6550772669
3 13.0490773814 12.5711656349
4 14.2121678724 13.6708033157
5 15.241296515 14.7290917967
6 15.9424957668 15.4741208116
7 16.7149132079 16.1630275368
8 17.5856712535 17.0049508553
9 18.4353515204 18.0324734616
10 18.8232525211 18.4709516348
11 19.2447137566 18.8619694268
12 19.5430023458 19.2203253937
13 19.8348917305 19.4902848051
14 20.1329059648 19.8638836864
